In [ ]:
from manim import *
import random
import numpy as np

class vector(Scene):
    def construct(self):
        # ---------- 标题 ----------
        t2 = Text("支持向量机", font="Microsoft YaHei", font_size=70, color=GREEN_C)
        t2.move_to(UP)
        self.add(t2)
        self.wait(1)

        # ---------- 坐标系 ----------
        axes1 = Axes(
            x_range=[0, 5, 1],
            y_range=[0, 5, 1],
            x_length=5,
            y_length=5,
            tips=True,
        ).move_to(DOWN)

        # ---------- 两类点 ----------
        cluster1_left = VGroup()
        cluster2_left = VGroup()
        random.seed(46)
        for _ in range(10):
            x1 = 1.5 + random.uniform(-1.0, 0.8)
            y1 = 1.5 + random.uniform(-0.9, 1.0)
            dot1 = Dot(axes1.coords_to_point(x1, y1), color=BLUE)
            cluster1_left.add(dot1)

            x2 = 3.5 + random.uniform(-1, 1)
            y2 = 3.5 + random.uniform(-0.9, 1)
            dot2 = Dot(axes1.coords_to_point(x2, y2), color=RED)
            cluster2_left.add(dot2)

        gg1 = VGroup(axes1, cluster1_left, cluster2_left)
        gg1.shift(DOWN*1.1).set_opacity(0)
        self.play(
            t2.animate.shift(UP*1.6),
            gg1.animate.shift(UP*1.1).set_opacity(1),
            run_time=1.5
        )
        self.wait(2)

        # --------- 点整体放大缩小 ----------
        self.play(
            cluster1_left.animate.scale(1.5),
            cluster2_left.animate.scale(1.5),
            run_time=1
        )
        self.play(
            cluster1_left.animate.scale(2/3),
            cluster2_left.animate.scale(2/3),
            run_time=1
        )
        self.wait(2)

        # ---------- 初始黄线 (辅助显示) ----------
        p1 = axes1.coords_to_point(2.4, 4.5)
        p2 = axes1.coords_to_point(2.6, 0)
        l1 = Line(p1, p2, color=YELLOW_D, stroke_width=3)
        self.play(FadeIn(l1))
        # ---------- 初始垂线 ----------
        def project_to_infinite_line(point, line):
            a, b = line.get_start(), line.get_end()
            dir_vec = b - a
            dir_vec /= np.linalg.norm(dir_vec)
            t = np.dot(point - a, dir_vec)
            return a + t*dir_vec
        # 找最近蓝点和红点（相对 l1）
        pp = axes1.coords_to_point(2.4, 3)
        nearest_blue = min(cluster1_left, key=lambda d: np.linalg.norm(d.get_center()-pp))
        nearest_red = min(cluster2_left, key=lambda d: np.linalg.norm(d.get_center()-pp))
        line_blue_init = Line(
            nearest_blue.get_center(),
            project_to_infinite_line(nearest_blue.get_center(), l1),
            color=YELLOW, stroke_width=2
        )
        line_red_init = Line(
            nearest_red.get_center(),
            project_to_infinite_line(nearest_red.get_center(), l1),
            color=YELLOW, stroke_width=2
        )
        # self.play(FadeIn(line_blue_init), FadeIn(line_red_init))
        # ---------- 初始矩形 ----------
        # 计算黄线方向单位向量
        line_dir = l1.get_unit_vector()  # l1 是初始黄线
        line_perp = np.array([-line_dir[1], line_dir[0], 0])  # 垂直于黄线的向量
        # 蓝色矩形：从黄线向左延伸到 nearest_blue
        blue_vec = nearest_blue.get_center() - l1.get_center()
        proj_len_blue = np.dot(blue_vec, line_perp)  # 投影到垂直方向
        rect_blue = Polygon(
            l1.get_start(),
            l1.get_end(),
            l1.get_end() + proj_len_blue*line_perp,
            l1.get_start() + proj_len_blue*line_perp,
            color=GREEN, fill_opacity=0.4, stroke_opacity=0
        )
        # 红色矩形：从黄线向右延伸到 nearest_red
        red_vec = nearest_red.get_center() - l1.get_center()
        proj_len_red = np.dot(red_vec, line_perp)
        rect_red = Polygon(
            l1.get_start(),
            l1.get_end(),
            l1.get_end() + proj_len_red*line_perp,
            l1.get_start() + proj_len_red*line_perp,
            color=GREEN, fill_opacity=0.4, stroke_opacity=0
        )
        self.play(FadeIn(rect_blue), FadeIn(rect_red))
        self.wait(2)

        scene_group = VGroup(axes1, cluster1_left, cluster2_left, l1, t2, rect_blue, rect_red)
        # 放大
        self.play(scene_group.animate.scale(4).shift(UP*0.6), run_time=1.5)
        self.wait(1)
        # ---------- 放大后加文字标注 ----------
        decision_text = Text("决策边界", font="Microsoft YaHei", font_size=40, color=YELLOW).move_to(UP).shift(LEFT*2+UP*2)
        margin_text = Text("间隔", font="Microsoft YaHei", font_size=32, color=WHITE).move_to(ORIGIN).shift(RIGHT*0.1+UP*1.7)
        self.play(Write(decision_text), Write(margin_text))
        self.wait(2)

        # 还原
        self.play(
            scene_group.animate.scale(1/4).shift(DOWN*0.6),
            FadeOut(decision_text, margin_text),
            run_time=1.5
        )

        pp = axes1.coords_to_point(2.4, 3)
        nearest_blue = min(cluster1_left, key=lambda d: np.linalg.norm(d.get_center()-pp))
        nearest_red = min(cluster2_left, key=lambda d: np.linalg.norm(d.get_center()-pp))

        # ---------- 在 (2.8,1) 加一个点 ----------
        new_dot = Dot(axes1.coords_to_point(2.8, 1), color=WHITE, radius=0.08)
        self.play(FadeIn(new_dot, scale=0.5))
        self.wait(1)
        self.play(new_dot.animate.set_color(RED), run_time=1.5)
        self.wait(1)
        # ---------- 删除初始线条 ----------
        self.play(FadeOut(l1, rect_blue, rect_red))
        self.play(new_dot.animate.set_color(BLUE), run_time=1.5)
        self.wait(1)

        # ---------- 动态旋转黄线 + 垂线更新 ----------
        angle_tracker = ValueTracker(0)
        initial_slope = -7.5  # 你想要的初始斜率
        theta0 = np.arctan(initial_slope)  # 转换成角度

        # 蓝红点中点
        mid_point = (nearest_blue.get_center() + nearest_red.get_center()) / 2

        # 黄线
        line_rot = always_redraw(lambda: Line(
            mid_point - 3*np.array([np.cos(angle_tracker.get_value() + theta0),
                                    np.sin(angle_tracker.get_value() + theta0), 0]),
            mid_point + 3*np.array([np.cos(angle_tracker.get_value() + theta0),
                                    np.sin(angle_tracker.get_value() + theta0), 0]),
            color=YELLOW_D, stroke_width=3
        ))

        # 投影到无限长直线
        def project_to_infinite_line(point, line):
            a, b = line.get_start(), line.get_end()
            dir_vec = b - a
            dir_vec /= np.linalg.norm(dir_vec)
            t = np.dot(point - a, dir_vec)
            return a + t*dir_vec

        # 蓝红垂线
        line_blue_rot = always_redraw(lambda: Line(
            nearest_blue.get_center(),
            project_to_infinite_line(nearest_blue.get_center(), line_rot),
            color=YELLOW, stroke_width=2
        ))
        line_red_rot = always_redraw(lambda: Line(
            nearest_red.get_center(),
            project_to_infinite_line(nearest_red.get_center(), line_rot),
            color=YELLOW, stroke_width=2
        ))

        self.add(line_rot, line_blue_rot, line_red_rot)

        # ---------- 动画：从初始角度旋转到最大间隔方向 ----------
        self.play(angle_tracker.animate.set_value(PI/4.35), run_time=3)
        self.wait(2)
        # ---------- 旋转结束后，解除 always_redraw ----------
        # 生成固定线条，替代动态线条
        line_rot_fixed = Line(line_rot.get_start(), line_rot.get_end(), color=YELLOW_D, stroke_width=3)
        line_blue_fixed = Line(line_blue_rot.get_start(), line_blue_rot.get_end(), color=YELLOW, stroke_width=2)
        line_red_fixed = Line(line_red_rot.get_start(), line_red_rot.get_end(), color=YELLOW, stroke_width=2)

        # 移除动态线条，添加固定线条
        self.remove(line_rot, line_blue_rot, line_red_rot)
        self.add(line_rot_fixed, line_blue_fixed, line_red_fixed)

        # 放大时使用固定线条，不会再跳动
        rot_group_fixed = VGroup(axes1, cluster1_left, cluster2_left, line_rot_fixed, line_blue_fixed, line_red_fixed, t2, new_dot)
        self.play(rot_group_fixed.animate.scale(4).shift(UP*1), run_time=1.5)
        # ---------- 旋转后矩形 ----------

        # 旋转后黄线方向单位向量
        line_dir_rot = line_rot_fixed.get_unit_vector()
        line_perp_rot = np.array([-line_dir_rot[1], line_dir_rot[0], 0])  # 垂直于黄线的向量

        # 蓝色矩形：从黄线向最近蓝点延伸
        blue_vec_rot = nearest_blue.get_center() - line_rot_fixed.get_center()
        proj_len_blue_rot = np.dot(blue_vec_rot, line_perp_rot)
        rect_blue_rot = Polygon(
            line_rot_fixed.get_start(),
            line_rot_fixed.get_end(),
            line_rot_fixed.get_end() + proj_len_blue_rot*line_perp_rot,
            line_rot_fixed.get_start() + proj_len_blue_rot*line_perp_rot,
            color=GREEN, fill_opacity=0.4, stroke_opacity=0
        )

        # 红色矩形：从黄线向最近红点延伸
        red_vec_rot = nearest_red.get_center() - line_rot_fixed.get_center()
        proj_len_red_rot = np.dot(red_vec_rot, line_perp_rot)
        rect_red_rot = Polygon(
            line_rot_fixed.get_start(),
            line_rot_fixed.get_end(),
            line_rot_fixed.get_end() + proj_len_red_rot*line_perp_rot,
            line_rot_fixed.get_start() + proj_len_red_rot*line_perp_rot,
            color=GREEN, fill_opacity=0.4, stroke_opacity=0
        )

        self.play(FadeIn(rect_blue_rot), FadeIn(rect_red_rot))
        self.wait(2)

        decision_text = Text("决策边界", font="Microsoft YaHei", font_size=40, color=YELLOW
        ).move_to(UP).shift(LEFT*2.4+UP*2.2) 
        margin_text = Text("间隔", font="Microsoft YaHei", font_size=32, color=WHITE
        ).move_to(ORIGIN).shift(LEFT*1.6+UP*1.3)
        ttt = Text("支持向量", font="Microsoft YaHei", font_size=32, color=RED_C
        ).move_to(ORIGIN).shift(RIGHT*1.9+UP*0.9) 
        ttt2 = Text("支持向量", font="Microsoft YaHei", font_size=32, color=BLUE_C
        ).move_to(ORIGIN).shift(LEFT*2.0+DOWN*2.7) 
        self.play(Write(decision_text), Write(margin_text),Write(ttt), Write(ttt2))
        self.wait(4)